In [1]:
var {
    fetchData,
    getSummarizedParameters,
    groupEachStage,
    palette,
    integratedLiquidLine,
    Plotly
} = require("./show-magma-model")

var funcTools = require("../../jslib/funcTools")

var fontMaker = (fontSize) => {
    return {
        family: 'sans-serif',
        size: fontSize,
        color: '#000'
    }
}

var middiumSize = {
    width : 800,
    height : 600
}

var largeSize = {
    width : 1000,
    height : 750
}

var legend= (normalOrder = true) => {
    return {
        legend: {
            traceorder : (normalOrder)? "normal" : "reversed" ,
            font: fontMaker(20)
        }
    }
    
}


var spread = (...objs) => [...objs].reduce((a,b)=>Object.assign(a,b),{})


var axis = (direction,title,range) => {
    const obj = {}
    obj[direction+"axis"] = {
        title : title, 
        titlefont:fontMaker(24),
        tickfont : fontMaker(18)
    };
    if (range !== undefined) obj[direction+"axis"].range = range;
    return obj
}

Directory

```
/diffusion
/phase
/ptmcmc
    /model
    /tool
    /result
    
```

In [2]:
var resultPath = "../result/1002-9-1C-opx1/2018-0719/"
var dataPath = "../model/"

var lavaPath = ""

In [3]:
var {
    meta,
    data,
    error,
    option,
    summary
} = fetchData(resultPath,dataPath)

var {
      model,
      parameters,
      updateCondition,
      constrain,
      magma
    } = require("../"+meta.model)(option)

var cmap = palette("tol-rainbow", parameters.length)


Summary column [ '', 'parameter', 'mean', 'stdev', 'mode', 'max_P' ]


In [5]:
var summarizedParameters = getSummarizedParameters(summary, "mean", parameters);
var parametersEachStage = groupEachStage(summarizedParameters)

summarizedParameters[0]

{ MgN_beforeCrystallization: '92.25421308675995',
  growth_stoichiometry_orthopyroxene: '0.2774050116828863',
  growth_stoichiometry_spinel: '0.044436804166669036',
  MgN_beforeMixing: '91.79478690554795',
  mixing_stoichiometry_orthopyroxene: '0.4500487909578216',
  mixing_stoichiometry_spinel: '0.04162039718272662',
  log10_tau: '8.031734297242306' }

In [6]:
Plotly([
    {
        y : funcTools.zipWith((x,y)=>1-x-y)
            (parametersEachStage.growth_stoichiometry_orthopyroxene)
            (parametersEachStage.growth_stoichiometry_spinel),
        name : "Olivine",
        type:"bar"
    },
    {
        y : parametersEachStage.growth_stoichiometry_orthopyroxene,
        name : "Orthopyroxene",
        type:"bar"
    },
    {
        y : parametersEachStage.growth_stoichiometry_spinel,
        name : "Spinel",
        type:"bar"
    }
],
spread(
    middiumSize,
    legend(),
    axis("x", "Stage number"),
    axis("y", "Stoichiometry", [0,1]),
    {
        barmode: "stack",
        title : "Stoichiometry in crystal growth"
    }
))

In [7]:
Plotly([
    {
        y : funcTools.zipWith((x,y)=>1-x-y)
            (parametersEachStage.mixing_stoichiometry_orthopyroxene)
            (parametersEachStage.mixing_stoichiometry_spinel),
        name : "Olivine",
        type:"bar"
    },
    {
        y : parametersEachStage.mixing_stoichiometry_orthopyroxene,
        name : "Orthopyroxene",
        type : "bar"
    },
    {
        y : parametersEachStage.mixing_stoichiometry_spinel,
        name : "Spinel",
        type : "bar"
    }
],
spread(
    middiumSize,
    legend(),
    axis("x", "Stage number"),
    axis("y", "Mass fraction", [0,1]),
    {
        barmode: "stack",
        title : "Mass fraction in approximating mixing",
    }
)
)


In [8]:
Plotly([
    {
        y : parametersEachStage.log10_tau
    }
],
spread(
    middiumSize,
    legend(),
    axis("x", "Stage number"),
    axis("y", "log10 net time scale of diffusion"),
    {
        title : ""
    }
)
)

In [9]:
var modeled = model(summarizedParameters,data)
var ndProfiles = magma.diffusionProfiles.orthopyroxene;

var ndFe_Mg = ndProfiles.Fe_Mg.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Fe_Mg.map(v=>100/(1+v)),
        name : "Model: diffused"
    },
     {
        x : ndFe_Mg.x,
        y : ndFe_Mg.c.map(v=>100/(1+v)),
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Fe_Mg.map(v=>100/(1+parseFloat(v))),
        mode : "markers",
        name : "Observed"
    }

],
spread(
    middiumSize,
    legend(),
    axis("x","Radius"),
    axis("y","Mg#")
)
)

In [10]:
var modeled = model(summarizedParameters,data)
var ndProfiles = magma.diffusionProfiles.orthopyroxene;

var ndFe_Mg = ndProfiles.Fe_Mg.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Fe_Mg,
        name : "Model: diffused"
    },
     {
        x : ndFe_Mg.x,
        y : ndFe_Mg.c,
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Fe_Mg,
        mode : "markers",
        name : "Observed"
    }

],
spread(
    middiumSize,
    legend(),
    axis("x","Radius"),
    axis("y","Fe/Mg")
))

In [11]:
var ndCr2O3 = ndProfiles.Cr2O3.notDiffusedProfile.get()

Plotly([
    {
        x : modeled.x,
        y : modeled.Cr2O3,
        name : "Model: diffused"
    },
    {
        x : ndCr2O3.x,
        y : ndCr2O3.c,
        name : "Model: original"
    },
    {
        x : data.x,
        y : data.Cr2O3,
        mode : "markers",
        name : "Observed"
    }

],
spread(
    middiumSize,
    legend(),
    axis("x","Radius"),
    axis("y","Cr2O3")
))

In [12]:

var liquidLine = integratedLiquidLine(magma,cmap)

//console.log(liquidLine("SiO2","MgO"))


In [13]:

Plotly(liquidLine("SiO2","MgO"),
spread(
    largeSize,
    legend(false),
    axis("x","SiO2"),
    axis("y","MgO")
)      
)


In [14]:
Plotly(liquidLine("SiO2","Cr2O3"),
spread(
    largeSize,
    legend(false),
    axis("x","SiO2"),
    axis("y","Cr2O3")
))


In [15]:

Plotly(liquidLine("SiO2","Al2O3"),
spread(
    largeSize,
    legend(false),
    axis("x","SiO2"),
    axis("y","Al2O3")
)      
)